# PHY5001 Assessment 2
##  Grover's algorithm 

This is the Jupyter Notebook which accompanies the pdf guide for Assignment 2, which focuses on Grover's Algorithm. Be advised that you should read the pdf as well, as there are questions which you will need to answer.

### Table of Contents
This hyperlinked table of contents may help reduce how much scrolling around you need to do to find the appropriate cells

- [Task 2: Selector](#Task2_cell)
    - [(a) GateSwitch](#Task2a_cell)
    - [(b) SelectorSix](#Task2b_cell)

    
- [Task 3: Amplifier](#Task3_cell)


- [Task 4: Putting together Grover's algorithm](#Task4_cell)


- [Task 5: Comparison of classical and quantum search](#Task5_cell)
    - [(a) RandomiseSix](#Task5a_cell)
    - [(b) Plotting comparison](#Task5Plot_cell)
    
    
- [Testing Functions](#Testing)
    - [Task 2 Testing](#Task2T_cell)
    - [Task 3 Testing](#Task3T_cell)
    - [Task 4 Testing](#Task4T_cell)
    - [Task 5 Testing](#Task5T_cell)

## Housekeeping
The cells below will import the appropriate packages you need for this assignment, assuming they are installed, and apply particular configuration settings for `DumpMachine`. You will need to run these cells each time you open this file to start work, as without them your functions below will not run as intended. 

In [19]:
import qsharp
import numpy as np
import matplotlib.pyplot as plt

In [20]:
%%qsharp
open Microsoft.Quantum.Diagnostics;

In [24]:
%config dump.basisStateLabelingConvention="BitString" 
%config dump.phaseDisplayStyle="NumberOnly"

## Task 2: Selector

<a id='Task2_cell'></a>

In this task, you will write code to set up the **Selector** component of Grover's algorithm. You will need to write two functions to build this Selector:
 - A function **GateSwitch** which applies Pauli-X gates to qubits in a 6 qubit array, using a Boolean array input to configure which qubits will be modified by the Pauli-X gates
 - A function **SelectorSix** which takes a 6 qubit array input and:
   - applies GateSwitch
   - applies a controlled-Z gate to the entire array, with the 6th qubit in the array being the target
   - applies GateSwitch again


<a id='Task2a_cell'></a>
In the cell below, write your code for **GateSwitch**.

*Hint: You will likely need to include a `for` loop and an `if` condition to make this work.*

In [ ]:
%%qsharp
operation GateSwitch(target: Qubit[], pattern: Bool[]) : Unit {
    mutable i = 0; 
    for q in target {
        if pattern[i] {
            X(q); 
        }
        i += 1; 
    }    
}

<a id='Task2b_cell'></a>
In the cell below, write your code for **SelectorSix**.

*Hint: Remember to use indices to indicate which qubit in the array you are referring to.*

In [ ]:
%%qsharp
operation SelectorSix(target: Qubit[], pattern: Bool[]) : Unit {
    GateSwitch(target, pattern); 
    Controlled Z(target[0..4],target[5]); 
    GateSwitch(target, pattern); 
}

## Task 3: Amplifier

<a id='Task3_cell'></a>

In this task, you will write code to set up the **Amplifier** component of Grover's algorithm. You will need to write one function to build this Amplifier:
 - A function **AmplifySix** which applies which applies the appropriate gates to a 6 qubit array, with the target of the controlled-Z gate being the 6th qubit in the array.
 
 
In the cell below, write your code for **AmplifySix**

*Hint: Remember to use indices to indicate which qubit in the array you are referring to. In Workshops you have seen the tools needed for at least 3 different approaches to writing AmplifySix - you may find it useful to use two `for` loops for part of this, but do not have to use any*

In [ ]:
%%qsharp
operation AmplifySix(target: Qubit[]) : Unit {
    ApplyToEach(H, target); 
    ApplyToEach(X, target); 
    Controlled Z(target[0..4],target[5]);
    ApplyToEach(X, target); 
    ApplyToEach(H, target); 
}

<a id='Task4_cell'></a>
## Task 4: Putting together Grover's algorithm

In this task, you will write:
- A function **SixQGroverIteration** which will apply SelectorSix and AmplifySix as required for a single iteration of Grover's algorithm
- A function **SixQGrovers** which will apply SixQGroverIteration multiple times in a row, as determined by an input `repeats`

<a id='Task4a_cell'></a>
In the cell below, write your code for **SixQGroverIteration**.

In [ ]:
%%qsharp
operation SixQGroverIteration(target: Qubit[], pattern: Bool[]) : Unit {
    SelectorSix(target, pattern); 
    AmplifySix(target); 
}

In the cell below, write your code for **SixQGrovers**

*Hint: One way to approach this is to write a `for` loop which will apply **SixQGroverIteration** on each repeat*

In [ ]:
%%qsharp
operation SixQGrovers(target: Qubit[], pattern: Bool[], repeats: Int) : Unit {
    for i in 0..repeats {
        SixQGroverIteration(target, pattern); 
    }
}

<a id='Task5_cell'></a>
## Task 5: Comparison of classical and quantum search

Now that we have a function that performs a single iteration of Grover's algorithm, let's work towards comparing the performance of quantum search against classical search. To do so, you will need to write:
- A function **RandomiseSix**, which should use Hadamard gates on all 6 qubits in a 6 qubit input to create a completely randomised quantum superposition, in order to allow a fair test of the algorithm
- Plotting code for comparing the performance of classical and quantum search algorithms


<a id='Task5a_cell'></a>
In the cell below, write your code for **RandomiseSix**.

*Hint: Remember to use indices to indicate which qubit in the array you are referring to.*

In [ ]:
%%qsharp
operation RandomiseSix(target: Qubit[]) : Unit {

    for X in 0..5 {
      H(target[X]);        
    } 
    
}

The cell below contains a function which will combine your RandomiseSix with your SixQGrovers. This will allow you to simulate the action of Grover's algorithm for searching. Assuming you have checked that your functions above work correctly, this should not need any adjustments in order to run. 

In [31]:
%%qsharp
operation SixGroverRun(repeats: Int, pattern: Bool[]) : Unit {
    use q = Qubit[6];
    RandomiseSix(q);
    SixQGrovers(q, pattern, repeats);
DumpMachine();
ResetAll(q);        
}

In the cell below:
- run a single iteration (eg. repeats = 1) of SixGroverRun for any 6 qubit Boolean pattern you choose - this will be fed into the selector. 
- Make note of the probability of measuring the state you have set the selector for.

Repeat the above, increasing the number of repeats from 1 to 20 in steps of 1, recording the probability of measuring the state you have selected at each point.

*Note: You may notice that the probability changes in an unexpected manner after 6 repeats - this is not an error!*

In [ ]:
%%qsharp

// Example: Call SixGroverRun with 2 repeats and pattern 101010:
SixGroverRun(2, [true,false,true,false,true,false])


<a id='Task5Plot_cell'></a>
Now for the comparison between classical and quantum search. Six qubits are capable of representing, at most, 64 ($2^{6}$) different objects simultaneously. For a fair comparison, we need to consider how many attempts it would take a classical computer to determine whether a particular item appears on a list. For a list with $N$ items, the probability $P$ of finding a particular item within $t$ attempts is:

$$P = 1 - (1 - \frac{1}{N})^t$$

In the cell below, write code which:
- builds an array `P_quant` out of the probability values you recorded above when running Grover's algorithm
- creates a linspace `t_quant` with 20 values, starting at 1 and ending at 20
- creates a linspace `t_class` with 250 values, starting at 1 and ending at 250
- calculate the classical probability `P_class` at each value of `t_class`, assuming $N = 64$ and using the equation above
- Make a figure which shows `P_class` plotted against `t_class`, and `P_quant` plotted against `t_quant` with appropriate axis labels, legend labels and title


Copy the image of the figure into your Assignment 1 document (by right-clicking on it and choosing "copy image as" or "save image as", and answer the questions in the Assignment 1 pdf which are about this plot

In [ ]:
P_quant = [X, Y, Z, A, B, C, D]
t_quant = np.linspace(A, B, C) 

t_class = np.linspace()
P_class = (1 - (MATH)**t_class)


# Plotting - Line/points



## Testing
<a id='Testing'></a>
This section contains scripts that will allow you to check whether functions you have written above are working as intended. Note that this is not a guarantee - you should still check the code you write to be sure that it is applying logic gates in the sequence that you intend - but most common errors will be caught with these checks.

<a id='Task2T_cell'></a>
### Task 2 Testing

The cells below will allow you to test that **GateSwitch** is working as intended. You must first run the cell which builds the function **TestGateSwitch**, and then the simulation operation beneath it.

If your **GateSwitch** is working correctly, then running the function **TestGateSwitch** will produce a ``DumpMachine`` result where there is a 25% probability of measuring the states $|110101\rangle$, $|110001\rangle$, $|100101\rangle$, and $|100001\rangle$. 



In [ ]:
%%qsharp
operation TestGateSwitch() : Unit {
use q = Qubit[6];
    H(q[1]);
    H(q[3]);
let testpattern = [true, true,false,true,false,true];   
GateSwitch(q,testpattern);        
DumpMachine();    
ResetAll(q);     
}

In [ ]:
TestGateSwitch.simulate()

The cells below will allow you to test that **SelectorSix** is working as intended. You must first run the cell which builds the function **TestSelectorSix**, and then the simulation operation beneath it.

If your **SelectorSix** is working correctly, then running the function **TestSelectorSix** will produce a `DumpMachine` result where every state has a 1.5625% probability of being measured, and the state $|001010\rangle$ has a phase of 3.1416.

In [ ]:
%%qsharp
operation TestSelectorSix() : Unit {
  use q = Qubit[6];  
    H(q[0]);
    H(q[1]);
    H(q[2]);
    H(q[3]);
    H(q[4]);
    H(q[5]);
let testpattern = [true,true,false,true,false,true];    
SelectorSix(q,testpattern);    
DumpMachine();    
ResetAll(q);    
}

In [ ]:

qsharp.eval(f"TestSelectorSix()")

<a id='Task3T_cell'></a>
### Task 3 Testing

The cells below will allow you to test that **AmplifySix** is working as intended. You must first run the cell which builds the function **TestAmplifySix**, and then the simulation operation beneath it.

If your **AmplifySix** is working correctly, then running the function **TestAmplifySix** will produce a `DumpMachine` result where states $|000100\rangle$ and $|100100\rangle$ have a 43.9453% chance of being measured, and a phase of 0. All other states will have a 0.1953% chance of being measured, and a phase of 3.1416.


In [ ]:
%%qsharp
operation TestAmplifySix() : Unit {
use q = Qubit[6];
H(q[0]);
X(q[3]);    
Z(q[5]);
AmplifySix(q);    
DumpMachine();    
ResetAll(q);          
}

In [ ]:
TestAmplifySix.simulate()

<a id='Task4T_cell'></a>
### Task 4 Testing

The cells below will allow you to test that **SixQGroverIteration** is working as intended. You must first run the cell which builds the function **TestSixQGroverIteration**, and then the simulation operation beneath it.

If your **SixQGroverIteration** is working correctly, then running the function **TestSixQGroverIteration** will produce a `DumpMachine` result where states $|001000\rangle$ and $|011000\rangle$ have a 43.9453% chance of being measured, and a phase of 0. All other states will have a 0.1953% chance of being measured, and a phase of 3.1416.

In [ ]:
%%qsharp
operation TestSixQGroverIteration() : Unit {
 use q = Qubit[6]; 
  let testpattern = [true,true,false,true,false,true];   
H(q[1]);
X(q[2]);    
Z(q[0]);    
SixQGroverIteration(q,testpattern);    
DumpMachine();
ResetAll(q);
    
    
}

In [ ]:
#TestSixQGroverIteration.simulate()

qsharp.eval("TestSixQGroverIteration")


<a id='Task5T_cell'></a>
### Task 5 Testing

In [ ]:
%%qsharp
operation TestRandomiseSix() : Unit {
use q = Qubit[6];
RandomiseSix(q);
DumpMachine();
ResetAll(q);
}

In [ ]:
TestRandomiseSix.simulate()